In [ ]:
import os
import pandas as pd
from matplotlib import pyplot as plt

%matplotlib inline

In [ ]:
# Google colab: 데이터 경로 설정
path = '../data/ml-latest-small/'

In [ ]:
os.listdir(path)

In [ ]:
ratings_df = pd.read_csv(os.path.join(path + 'ratings.csv'), encoding='utf-8')
tags_df = pd.read_csv(os.path.join(path + 'tags.csv'), encoding='utf-8')
movies_df = pd.read_csv(os.path.join(path + 'movies.csv'), index_col='movieId', encoding='utf-8')

* timestamp는 영화에 대한 평점을 기록한 시점. 영화가 처음 출시한 시점이 아니다

In [ ]:
print(ratings_df.shape)
print(ratings_df.head())

In [ ]:
print(tags_df.shape)
print(tags_df.head())

In [ ]:
print(movies_df.shape)
print(movies_df.head())

### 평점 데이터의 기초 통계량

In [ ]:
n_unique_users = len(ratings_df['userId'].unique())
print(n_unique_users)

In [ ]:
n_unique_movies = len(ratings_df['movieId'].unique())
print(n_unique_movies)

In [ ]:
print('평점의 평균: ', ratings_df['rating'].mean())
print('평점의 표준편차: ', ratings_df['rating'].std())

In [ ]:
ratings_df.info()

In [ ]:
ratings_df.describe()

In [ ]:
# nan 값이 있는지 확인한다
ratings_df.isnull().sum()

In [ ]:
ratings_df[['userId','movieId','rating']].hist()

### Pandas의 GroupBy 사용하기

In [ ]:
ratings_df.groupby(['userId', 'rating']).size()

* userId와 rating을 기준으로 기초 통계량

In [ ]:
ratings_df.groupby(['userId']).mean()

In [ ]:
userid_rating_df = pd.DataFrame({'count': ratings_df.groupby(['userId', 'rating']).size()})
userid_rating_df = userid_rating_df.reset_index()
userid_rating_df.head(10)

In [ ]:
user_info = ratings_df.groupby('userId')['movieId'].count()

In [ ]:
user_info.describe()

In [ ]:
import seaborn as sns

In [ ]:
# user가 몇개의 영화에 대해서 rating을 했는지를 볼 수 있는 분포(그래프)
sns.distplot(user_info.values)

### user가 평균적으로 준 평점과 평점을 준 영화의 수

In [ ]:
stats_df = pd.DataFrame({
    'movie_count': ratings_df.groupby('userId')['movieId'].count(),
    'rating_avg': ratings_df.groupby('userId')['rating'].mean(), 
'rating_std': ratings_df.groupby('userId')['rating'].std()})

print(stats_df.shape)
print(stats_df.head())

### rating이 많은 영화(다시 말해, 사람들이 관심 많은 영화)

* 많이 본 영화일수록 평점이 좋다? 

In [ ]:
movieid_user_df = pd.DataFrame({
    'num_users_watch': ratings_df.groupby('movieId')['userId'].count(),
    'avg_ratings': ratings_df.groupby('movieId')['rating'].mean(),
    'std_ratings': ratings_df.groupby('movieId')['rating'].std()
})
movieid_user_df = movieid_user_df.reset_index()
print(movieid_user_df.shape)
print(movieid_user_df.head(10))

In [ ]:
movieid_user_df.sort_values(by='num_users_watch', ascending=False)

In [ ]:
# long tail economy
movieid_user_df['num_users_watch'].hist()

In [ ]:
# 1번 또는 1명만 평점을 준 영화
movieid_user_df['movieId'][movieid_user_df.num_users_watch == 1].count()


In [ ]:
# 3번 미만의 영화
movieid_user_df['movieId'][movieid_user_df.num_users_watch < 3].count()

### 평점이 높은 영화(장르), 평점을 많이 받은 영화(장르)

In [ ]:
# 영화 정보를 활용
movies_df.head()

In [ ]:
# 평점을 많이 받은 영화 
ratings_count_df = ratings_df.groupby('movieId')['userId'].count()
ratings_count_df.head()

In [ ]:
df = pd.DataFrame({
    'ratings_count': ratings_df.groupby('movieId')['userId'].count(),
})

In [ ]:
df['movie_name'] = df.apply(lambda x: movies_df['title'].loc[x.index])

In [ ]:
df.head(10)

In [ ]:
df.sort_values(by='ratings_count', ascending=False)

### 특정 영화의 평점 분포

* movieId :  356, Forrest Gump(1994)
* movieId : 2571, Matrix, The (1999)

In [ ]:
ratings_df[ratings_df.movieId == 356]['rating'].hist()

In [ ]:
ratings_df[ratings_df.movieId == 2571]['rating'].hist()

### 특정 유저가 주는 평점과 실제 영화에 대한 평점 분포가 비슷할까? 

* 나는 일반적인 영화 관람객일까? 특이한 취향을 갖고있는 사람일까? 

In [ ]:
ratings_df[(ratings_df.movieId == 356) & (ratings_df.rating < 2)]

In [ ]:
ratings_df[(ratings_df.movieId == 2571) & (ratings_df.rating < 2)]

In [ ]:
ratings_df[ratings_df.userId == 76]['rating'].hist()